# Shear profile of A360 using Anacal - Manual

Prakruth Adari, Xiangchong Li, Anja von der Linden, Anthony Englert\
LSST Science Piplines version: Weekly 2025_26\
Container Size: large

This notebook is a condensed set of code to obtain a shear profile of Abell 360 using Anacal shear measurements. The main steps are:

- Loading the relevant object catalogs (all tracts and patches needed) using the butler
- Loading in Anacal catalogs
- Matching between the two (using RA/DEC + _i_-magnitude)
- Ca
- Load in calibration (using the `gen_hsc_calibration` script)
- Shear profile
  
Most steps start with loading in data from the previous step so each step usually ends with writing data locally, this means we can quickly apply cuts to the same catalog and re-calibrate without having to re-query from the Butler. 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import os, sys
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import scipy.interpolate as interpolate
import scipy.integrate as integrate
import scipy.stats as stats
from matplotlib import cm
from astropy.table import Table, join, vstack
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS


# Familiar stack packages
from lsst.daf.butler import Butler
import lsst.geom as geom

import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

z_cl = 0.22

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)
rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

In [ ]:
arcsec = 1/60**2
rng = np.random.default_rng()
omega_m = .31
omega_de= .69
omega_r = 0
H0 = 70 # km/s/Mpc
c = 3e5 # km/s

Hz = lambda z : c/(H0  * np.sqrt((omega_de + omega_m * (1+z)**3 + omega_r * (1+z)**4)))
chi_dl = lambda z, z0=0 : integrate.quad(Hz, z0, z)[0]
Da = lambda z : chi_dl(z)/(1+z)
beta_r = lambda zl, zs : integrate.quad(Hz, zl, zs)[0]/integrate.quad(Hz, 0, zs)[0]

In [ ]:
# Load operation rehearsal data
# Can use obs_butler.registry.queryCollections to see available collections

flux_suffix = '_cModelFlux'
# obs_repo = '/repo/main'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0_rc5/DM-49865'

obs_repo = '/repo/dp1'
obs_collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'
obs_butler = Butler(obs_repo, collections=obs_collection)
obs_registry = obs_butler.registry

t_skymap = obs_butler.get('skyMap', skymap='lsst_cells_v1')

In [ ]:
cluster_coords = (37.86501659859067, 6.982204815599694)

## Load Butler Data

In [ ]:
# Position of the BCG for A360
ra_bcg = cluster_coords[0]
dec_bcg = cluster_coords[1]

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = t_skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
print(tp_dict)

## Precuts

In [ ]:
anacal_tables = []
version = "v5"
if version == "v1":
    mag_zero = 27
else:
    mag_zero = 31.4
for tid in tp_dict.keys():
    patches = tp_dict[tid]
    for p in patches:
        fname = f'/sdf/data/rubin/shared/cluster_commissioning/AnaCal/catalog_{version}/{tid}-{p}.fits'
        if os.path.isfile(fname):
            ana_table = Table.read(fname)
            mag = mag_zero - np.log10(ana_table["flux"]) * 2.5
            mask = (mag<23.5)
            ana_table = ana_table[mask]
            anacal_tables.append(ana_table)
        else:
            print(fname)
            
anacal_table = vstack(anacal_tables)
anacal_mag = mag_zero - np.log10(anacal_table['flux'] ) *2.5
anacal_table['mag'] = anacal_mag

In [ ]:
# prepare matching
anacal_coords = np.vstack((anacal_table['ra'], anacal_table['dec'])).T
anacal_tree = KDTree(anacal_coords)
# Load in pre-cuts data
with fits.open('./cluster_data/abell360_PRECUTS_DP1_gaap_cModel.fits') as hdul:
        data = hdul[1].data
        table = Table(data)

In [ ]:
table_coords = SkyCoord(table['coord_ra']*u.deg, table['coord_dec']*u.deg)
cluster_skycoord = SkyCoord(cluster_coords[0]*u.deg, cluster_coords[1]*u.deg)
sep = table_coords.separation(cluster_skycoord)
table['sep'] = sep.deg

In [ ]:
gi = table['g_gaap_mag'] - table['i_gaap_mag']
ri = table['r_gaap_mag'] - table['i_gaap_mag']
gr = table['g_gaap_mag'] - table['r_gaap_mag']

table['gi'] = gi
table['ri'] = ri
table['gr'] = gr

In [ ]:
wl_coords = np.vstack((table['coord_ra'], table['coord_dec'])).T
wl_tree = KDTree(wl_coords)
match_dist, match_ndx = anacal_tree.query(wl_coords)
mag_diffs = table['i_cModel_mag'] - anacal_table['mag'][match_ndx]
trial_match = match_dist < 1.5*arcsec
close_match = match_dist < 0.5 * arcsec

In [ ]:
#plt.hist(mag_diffs, range=(-2, 5), bins=100, label='All Match')
plt.hist(mag_diffs[trial_match], range=(-2, 2), bins=100, label="Near ($< 1.5''$) Match", histtype='step')
plt.hist(mag_diffs[close_match], range=(-2, 2), bins=100, label="Near ($< 0.5''$) Match", histtype='step')
plt.title("Comcam object $i$ mag - Anacal nearest match $i$ mag")
plt.legend()

In [ ]:
near_match = match_dist < 0.5*arcsec
good_anacal = near_match * (np.abs(mag_diffs) < 0.5)
print(np.sum(good_anacal), len(good_anacal))

table['i_anacal_goodflag'] = good_anacal

In [ ]:
mag_lim = 24

gi_table_line = (table['r_cModel_mag'] - 18) * (-0.2/5) + 1.9
ri_table_line = (table['r_cModel_mag'] - 18) * (-0.1/5) + .55
gr_table_line = (table['r_cModel_mag'] - 18) * (-0.25/5) + 1.35

gi_lims = (0.1, -.25)
ri_lims = (0.04, -0.08)
gr_lims = (0.1, -.15)

gi_redseq = np.logical_and.reduce((table['gi'] < gi_table_line + gi_lims[0], table['gi'] > gi_table_line + gi_lims[1], table['r_cModel_mag'] < mag_lim))
ri_redseq = np.logical_and.reduce((table['ri'] < ri_table_line + ri_lims[0], table['ri'] > ri_table_line + ri_lims[1], table['r_cModel_mag'] < mag_lim))
gr_redseq = np.logical_and.reduce((table['gr'] < gr_table_line + gr_lims[0], table['gr'] > gr_table_line + gr_lims[1], table['r_cModel_mag'] < mag_lim))


table['gi_redseq'] = gi_redseq
table['ri_redseq'] = ri_redseq
table['gr_redseq'] = gr_redseq

all_redseq = np.logical_and.reduce((table['gi_redseq'], table['gr_redseq'], table['ri_redseq']))
aggro_redseq = np.logical_or.reduce((table['gi_redseq'], table['gr_redseq'], table['ri_redseq']))
table['all_redseq'] = all_redseq
table['aggro_redseq'] = aggro_redseq

In [ ]:
ae1 = anacal_table['fpfs_e1']
ae2 = anacal_table['fpfs_e2']
lct = len(table)
wl_ae1 = np.ones(lct) * -99.
wl_ae2 = np.ones(lct) * -99.

wl_r1 = np.ones(lct) * -99.
wl_r2 = np.ones(lct) * -99.

for i in range(lct):
    if good_anacal[i]:
        mid = match_ndx[i]
        wl_ae1[i] = ae1[mid] * anacal_table["wdet"][mid]
        wl_ae2[i] = ae2[mid] * anacal_table["wdet"][mid]
        wl_r1[i] = anacal_table["fpfs_de1_dg1"][mid] * anacal_table["wdet"][mid] + anacal_table["dwdet_dg1"][mid] * anacal_table["fpfs_e1"][mid]
        wl_r2[i] = anacal_table["fpfs_de2_dg2"][mid] * anacal_table["wdet"][mid] + anacal_table["dwdet_dg2"][mid] * anacal_table["fpfs_e2"][mid]

table['anacal_fpfs_e1'] = wl_ae1
table['anacal_fpfs_e2'] = wl_ae2
table['anacal_fpfs_R1'] = wl_r1
table['anacal_fpfs_R2'] = wl_r2

In [ ]:
ae1 = anacal_table['e1']
ae2 = anacal_table['e2']
lct = len(table)
wl_ae1 = np.ones(lct) * -99.
wl_ae2 = np.ones(lct) * -99.

wl_r1 = np.ones(lct) * -99.
wl_r2 = np.ones(lct) * -99.

for i in range(lct):
    if good_anacal[i]:
        mid = match_ndx[i]
        wl_ae1[i] = ae1[mid] * anacal_table["wdet"][mid]
        wl_ae2[i] = ae2[mid] * anacal_table["wdet"][mid]
        wl_r1[i] = anacal_table["de1_dg1"][mid] * anacal_table["wdet"][mid] + anacal_table["dwdet_dg1"][mid] * anacal_table["e1"][mid]
        wl_r2[i] = anacal_table["de2_dg2"][mid] * anacal_table["wdet"][mid] + anacal_table["dwdet_dg2"][mid] * anacal_table["e2"][mid]

table['anacal_e1'] = wl_ae1
table['anacal_e2'] = wl_ae2
table['anacal_R1'] = wl_r1
table['anacal_R2'] = wl_r2

In [ ]:
# Apply cuts
filt = np.ones(len(table)).astype(bool)

filt &= (table['refExtendedness'] >= 0.5)
filt &= (table['sep'] < 0.5) 
# filt &= table['g_cModel_flag']== False
# filt &= table['r_cModel_flag']== False
# filt &= table['i_cModel_flag']== False
filt &= (table['i_cModel_mag'] <= 23)
# filt &= (table['i_cModel_mag'] >= 20)
# filt &= (table['i_cModelFlux']/table['i_cModelFluxErr'] >= 10)
filt &= (table['i_anacal_goodflag'])
# filt &= table['res'] >= 0.3
# filt &= table['i_blendedness'] <= 0.42

# filt &= (table['anacal_fpfs_e1']**2 + table['anacal_fpfs_e2']**2) <= 4
# filt &= (table['i_hsmShapeRegauss_sigma']<= 0.4) * (0 < table['i_hsmShapeRegauss_sigma'])
# filt &= table['i_iPSF_flag']==0
filt &= ~(table['all_redseq'])
print(f"After cuts: {np.sum(filt)}")
wl_table = table[filt]

# Profile

Requires `wl_table` to be defined from the [Calibration](#Calibration) Step.

In [ ]:
def anacal_get_tang_cross(e_t, e_x, sky_dist, bins, R1, R2, astropy=True, ci_level=.95):
    nb = len(bins) - 1
    tang_avg = np.zeros(nb)
    cross_avg = np.zeros_like(tang_avg)

    tang_err = np.zeros((nb, 2))
    cross_err = np.zeros_like(tang_err)

    bin_rs = []
    for i in range(nb):
        # bin_rs.append(bins[i])
        ndx_filt = (sky_dist > bins[i]) * (sky_dist < bins[i+1])
        bin_rs.append(np.where(ndx_filt)[0])
        if np.sum(ndx_filt) < 1:
            continue
            
        sample_t = e_t[ndx_filt]
        sample_x = e_x[ndx_filt]

        sample_r1 = np.mean(R1[ndx_filt])
        sample_r2 = np.mean(R2[ndx_filt])

        R_correction = (.5 * (sample_r1 + sample_r2)) 
        # R_correction = np.float64(1.)
        # R_correction = (.5 * (np.mean(R1) + np.mean(R2)))
        print(R_correction)
        
        ta = np.mean(sample_t)/R_correction
        xa = np.mean(sample_x)/R_correction
        te = stats.bootstrap([sample_t], np.mean, confidence_level=ci_level).confidence_interval
        xe = stats.bootstrap([sample_x], np.mean, confidence_level=ci_level).confidence_interval
        
        tang_avg[i] = ta
        cross_avg[i] = xa
        tang_err[i] = te/R_correction
        cross_err[i] = xe/R_correction

    return tang_avg, cross_avg, tang_err, cross_err, bin_rs

In [ ]:
cluster_coords = (37.865017, 6.982205)
# cluster_coords = (37.865017, 7.082205)
source_phi = np.arctan2(wl_table['coord_dec'] - cluster_coords[1], (cluster_coords[0] - wl_table['coord_ra'])*np.cos(np.deg2rad(cluster_coords[1])))
ang_dist = np.sqrt(((wl_table['coord_ra'] - cluster_coords[0]) * np.cos(np.deg2rad(cluster_coords[1])))**2 + (wl_table['coord_dec'] - cluster_coords[1])**2)
sky_distance = Da(.22) * ang_dist * (np.pi/180)

In [ ]:
bins_mpc = np.array([.3, .5, 1, 1.5, 2.27, 3.3, 6])
bin_mids = 1/2 * (bins_mpc[1:] + bins_mpc[:-1])

In [ ]:
trial_shear = wl_table['anacal_e1'] + 1.j*wl_table['anacal_e2']
cl_shear = trial_shear * -1*np.exp(-2j*source_phi)

wl_table['e_t'] = cl_shear.real
wl_table['e_x'] = cl_shear.imag

shear_cl = anacal_get_tang_cross(wl_table['e_t'], wl_table['e_x'], sky_distance, bins_mpc, wl_table['anacal_R1'], wl_table['anacal_R2'], astropy=False, ci_level=.68)

In [ ]:
trial_shear = wl_table['anacal_fpfs_e1'] + 1.j*wl_table['anacal_fpfs_e2']
cl_shear = trial_shear * -1*np.exp(-2j*source_phi)

wl_table['e_fpfs_t'] = cl_shear.real
wl_table['e_fpfs_x'] = cl_shear.imag

shear_cl_fpfs = anacal_get_tang_cross(wl_table['e_fpfs_t'], wl_table['e_fpfs_x'], sky_distance, bins_mpc, wl_table['anacal_fpfs_R1'], wl_table['anacal_fpfs_R2'], astropy=False, ci_level=.68)

In [ ]:
# moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")
# moo.set_cosmo(cosmo)
# moo.set_concentration(4)
# moo.set_mass(1.0e15)
# gt_z = moo.eval_reduced_tangential_shear(
#     rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
# )

In [ ]:
cmap = cm.coolwarm

# plt.plot(rproj, gt_z, label=r'NFW, $M_{200}$=$10^{15} M_{\odot}$, c=4, n(z)=SRD', ls=':')
plt.plot(bin_mids, shear_cl[0], '.', label='tangential - e', color=cmap(.05), alpha=0.25)
plt.vlines(bin_mids, shear_cl[2][:,0], shear_cl[2][:,1], color=cmap(0.05), alpha=0.25)
plt.plot(1.05*bin_mids, shear_cl[1], '.', label='cross - e', color=cmap(.95), alpha=0.25)
plt.vlines(1.05*bin_mids, shear_cl[3][:,0], shear_cl[3][:,1], color=cmap(0.95), alpha=0.25)

plt.plot(bin_mids*1.1, shear_cl_fpfs[0], '.', label='tangential - fpfs', color=cmap(.05), alpha=1)
plt.vlines(bin_mids*1.1, shear_cl_fpfs[2][:,0], shear_cl_fpfs[2][:,1], color=cmap(0.05), alpha=1)
plt.plot(1.15*bin_mids, shear_cl_fpfs[1], '.', label='cross - fpfs', color=cmap(.95), alpha=1)
plt.vlines(1.15*bin_mids, shear_cl_fpfs[3][:,0], shear_cl_fpfs[3][:,1], color=cmap(0.95), alpha=1)

plt.semilogx()
plt.axhline(0, ls='--', color='k', alpha=.5)

plt.ylim([-0.08,0.10])
plt.xlim([0.3,6.5])
plt.ylabel("Reduced shear")
plt.xlabel("R (Mpc)")
plt.legend(frameon=False, loc='upper right')

2nd bin at 0.75 Mpc so $\theta = .75 * \frac{180}{\pi} * \frac{1}{D_A(0.22)} \approx 0.06^\circ$ 

In [ ]:
print("Bin 1", 0.75 * 180/np.pi * 1/Da(0.22))
print("Bin 2", 1.25 * 180/np.pi * 1/Da(0.22))

In [ ]:
# From Shenming's CLMM demo on using HSC data
def apply_shear_calibration(e1_0, e2_0, e_rms, m, c1, c2, weight):
    R = 1.0 - np.sum(weight * e_rms**2.0) / np.sum(weight)
    m_mean = np.sum(weight * m) / np.sum(weight)
    c1_mean = np.sum(weight * c1) / np.sum(weight)
    c2_mean = np.sum(weight * c2) / np.sum(weight)
    print("R, m_mean, c1_mean, c2_mean: ", R, m_mean, c1_mean, c2_mean)

    g1 = (e1_0 / (2.0 * R) - c1) / (1.0 + m_mean)
    g2 = (e2_0 / (2.0 * R) - c2) / (1.0 + m_mean)

    return g1, g2

In [ ]:
calib_filename = 'a360_calib_all.fits'
postcuts_filename = 'abell360_POSTCUTS_all_DP1.fits'

In [ ]:
with fits.open(f'cluster_data/technote/{calib_filename}') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    table = Table(data)

with fits.open(f'cluster_data/technote/{postcuts_filename}') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    wl_table = Table(data)

In [ ]:
e_rms = table["ishape_hsm_regauss_derived_rms_e"]
m = table["ishape_hsm_regauss_derived_shear_bias_m"]
c1 = table["ishape_hsm_regauss_derived_shear_bias_c1"]
c2 = table["ishape_hsm_regauss_derived_shear_bias_c2"]
weight = table["ishape_hsm_regauss_derived_shape_weight"]
# weight = np.ones(len(c1))

In [ ]:
g1, g2 = apply_shear_calibration(wl_table['i_hsmShapeRegauss_e1'], wl_table['i_hsmShapeRegauss_e2'], e_rms, m, c1, c2, weight)

## Mass Aperture Statistics

Mass aperture statistics are one way of mapping the distribution of dark matter across a cluster, it's an integral statistic which convolves the observed shears with a filter optimized to match a given profile. For LoVoCCS (which I'll borrow here), we normally use a 'Schirmer filter' (Schirmer+04, Hetterscheidt+05, Schirmer+06, McCleary+18, Fu+22, Fu+24)

In [ ]:
def schirmer_filter(radius,aperture_size=8000,x_cut=0.15,a=6,b=150,c=47,d=50,*_):
    '''
    The Schirmer Filter, a filter which is optimized for detecting NFW-like structures in shear-fields.
    
    Args:
        radius: Numpy array; an array of radii to evaluate the filter on
        aperture_size: float-like; the 'schirmer-radius' of the filter
        x_cut: float-like; specifies the filter-sloap and sets the characteristic-scale of the filter to x_cut*smoothing
    
    Returns:
        Q; Numpy array; an array containing the filter evaluated at each radius
    
    '''
    
    x = radius/aperture_size
    Q = ( 1/( 1 + np.exp(a - b*x) + np.exp(-c + d*x)) )*( np.tanh(x/x_cut)/(x/x_cut) )
    return Q

In [ ]:
xs = np.linspace(0.01, 2, 101)
Qs = schirmer_filter(xs, aperture_size=1)

plt.plot(xs, Qs, '.')

In [ ]:
def compute_mass_map(x_grid,y_grid,x,y,g1,g2,weights,q_filter,filter_kwargs={}):
    '''
    This function computes the mass aperture-statistics at each point on a specified grid. Run quality-cuts, NaN filtering, etc. before this step!
    
    Args:
        x: Numpy array; an array of x-coordinates for each object
        y: Numpy array; an array of y-coordinates for each object
        x_grid: Numpy array; an NxM array of x-coordinates to sample the aperture-mass on
        y_grid: Numpy array; an NxM array of y-coordinates to sample the aperture-mass on
        g1; Numpy array; the shear g1 for each object
        g2; Numpy array; the shear g2 for each object
        weights: Numpy array; the weight for each object's shear
        q_filter; function; the filter-function used to compute Map
        kwargs; dict; kwargs passed to w_filter
    
    Returns:
        Map_E: Numpy array; an NxM array containing the E-mode aperture mass evaluated at each grid-point
        Map_B: Numpy array; an NxM array containing the B-mode aperture mass evaluated at each grid-point
        Map_V: Numpy array; an NxM array containing the variance in the aperture mass evaluated at each grid-point

    '''

    y_shape = len(y_grid[:,0])
    x_shape = len(x_grid[0,:])
    
    Map_E = np.zeros((y_shape,x_shape))
    Map_B = np.zeros((y_shape,x_shape))
    Map_V = np.zeros((y_shape,x_shape))
    
    if 'aperture_size' not in filter_kwargs:
        filter_area = np.pi * (8000)**2
    else:
        filter_area = np.pi * filter_kwargs['aperture_size']**2
    
    # an extra catch for an objects assigned NaN g1/g2 just in case
    # nan_catch = np.isfinite(g1) & np.isfinite(g2)
    # x = x[nan_catch]
    # y = y[nan_catch]
    # g1 = g1[nan_catch]
    # g2 = g2[nan_catch]
    # weights = weights[nan_catch]

    comp_shear = g1 + 1j*g2
    g_mag = g1**2 + g2**2
    
    for i in range(y_shape):
        for j in range(x_shape):
            delta_x = x_grid[j,i] - x
            delta_y = y_grid[j,i] - y
            radius = np.sqrt(delta_x**2 + delta_y**2)# * deg2pix_conv
            theta = np.arctan2(delta_y,delta_x)
            rotated_shear = comp_shear * -1*np.exp(-2j*theta)
            g_T = rotated_shear.real
            g_X = rotated_shear.imag
            # g_T = -g1*np.cos(2*theta) - g2*np.sin(2*theta)
            # g_X =  g1*np.sin(2*theta) - g2*np.cos(2*theta)
            # g_mag = g1**2 + g2**2

            filter_values = q_filter(radius,**filter_kwargs)
    

            weight_sum = np.sum(weights)
            
            Map_E[i,j] = np.sum(filter_values*g_T*weights)*filter_area/weight_sum
            Map_B[i,j] = np.sum(filter_values*g_X*weights)*filter_area/weight_sum
            Map_V[i,j] = np.sum( (filter_values**2)*g_mag*(weights**2) )*(filter_area**2)/(2*(weight_sum**2))
    
    return Map_E, Map_B, Map_V

### Global R

In [ ]:
ra_bcg, dec_bcg = (37.86501659859067, 6.982204815599694)

In [ ]:
# first, setup a coordinates for a flat-sky in x and y
# create a WCS and use the corresponding method to transform sky to px coords

# load wcs from astropy
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel

# also load skycoord for the conversion
from astropy.coordinates import SkyCoord

# build wcs centered on BCG
flat_wcs = WCS(naxis=2)
crval_sky = [ra_bcg*u.deg,dec_bcg*u.deg]
flat_wcs.wcs.crval = [ra_bcg,dec_bcg]
flat_wcs.wcs.crpix = [0,0] # assign 0,0 to the center, shouldn't matter
flat_wcs.wcs.cdelt = [-0.2/3600,0.2/3600] # match angular resolution of LSST, 0.2"
flat_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
flat_wcs.wcs.radesys = 'ICRS'
flat_wcs.wcs.equinox = 2000
flat_wcs.wcs.cd = [[-0.2/3600,0],[0,0.2/3600]]

# compute coords in the flat-sky
# coords = SkyCoord(ra=merged_cat_wl['coord_ra'][source_filt][to_use],dec=merged_cat_wl['coord_dec'][source_filt][to_use],unit='degree')
coords = SkyCoord(ra=wl_table['coord_ra'],dec=wl_table['coord_dec'],unit='degree')
x,y = skycoord_to_pixel(coords,wcs=flat_wcs)

# scale these up to units of degrees to match mass_map spacing and make writing the aperture easier
x = x * 0.2/3600
y = y * 0.2/3600

# for now I'll weight everything uniformly
weights = np.ones(len(x))

# Define an NxN grid centered on the cluster
# Spans 1-deg centered at the BCG
# scale so that its in pixel coordinates for computing
N = 51

mid_x = 0
mid_y = 0
x_grid_samples = np.linspace(mid_x-0.5,mid_x+0.5,N)
y_grid_samples = np.linspace(mid_y-0.5,mid_y+0.5,N)
y_grid,x_grid = np.meshgrid(y_grid_samples,x_grid_samples)

#### non-FPFS 

In [ ]:
# Using a global R for the first try:

global_R = 0.5 * (np.mean(wl_table['anacal_R1']) + np.mean(wl_table['anacal_R2']))
print(global_R)

weights = np.ones(len(x))
g1 = wl_table['anacal_e1'] / global_R
g2 = wl_table['anacal_e2'] / global_R

In [ ]:
# this takes a minute to run
e_ap,b_ap,v_ap = compute_mass_map(x_grid,y_grid,x,y,g1,g2,weights,schirmer_filter,filter_kwargs={'aperture_size':0.75})

In [ ]:
sky_box = np.array([[x_grid.max(), x_grid.min()],[y_grid.min(), y_grid.max()]])

In [ ]:
test_xs = np.linspace(-5, 5, num=1001)
standard = stats.norm(0,1)
normal_dist = standard.pdf(test_xs)

In [ ]:
plt.hist((b_ap/np.sqrt(v_ap)).flatten(), bins=51, label='B Mode', histtype='step', density=True)
plt.hist((e_ap/np.sqrt(v_ap)).flatten(), bins=51, label='E Mode', histtype='step', density=True)
plt.plot(test_xs, normal_dist, '--')
plt.legend();

In [ ]:
print(np.abs(e_ap/np.sqrt(v_ap)).max(), np.abs(b_ap/np.sqrt(v_ap)).max())

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(10,6))

ax = axs[0]
ax.set_title('E-Mode SN')
MapE = ax.imshow(e_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

ax = axs[1]
ax.set_title('B-Mode SN')
MapB = ax.imshow(b_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

# for xx in axs:
#     xx.plot(ra_bcg, dec_bcg, 'rx')
cbar = fig.colorbar(MapE, ax=axs,fraction=0.025)

fig.supxlabel("RA", y=0.14)
fig.supylabel("DEC", x=0.05)
#fig.savefig('ACO360_mass_map.png',dpi=480)
# woohoo, we have a signal!

#### FPFS 

In [ ]:
# first, setup a coordinates for a flat-sky in x and y
# create a WCS and use the corresponding method to transform sky to px coords

# load wcs from astropy
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel

# also load skycoord for the conversion
from astropy.coordinates import SkyCoord

# build wcs centered on BCG
flat_wcs = WCS(naxis=2)
crval_sky = [ra_bcg*u.deg,dec_bcg*u.deg]
flat_wcs.wcs.crval = [ra_bcg,dec_bcg]
flat_wcs.wcs.crpix = [0,0] # assign 0,0 to the center, shouldn't matter
flat_wcs.wcs.cdelt = [-0.2/3600,0.2/3600] # match angular resolution of LSST, 0.2"
flat_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
flat_wcs.wcs.radesys = 'ICRS'
flat_wcs.wcs.equinox = 2000
flat_wcs.wcs.cd = [[-0.2/3600,0],[0,0.2/3600]]

# compute coords in the flat-sky
# coords = SkyCoord(ra=merged_cat_wl['coord_ra'][source_filt][to_use],dec=merged_cat_wl['coord_dec'][source_filt][to_use],unit='degree')
coords = SkyCoord(ra=wl_table['coord_ra'],dec=wl_table['coord_dec'],unit='degree')
x,y = skycoord_to_pixel(coords,wcs=flat_wcs)

# scale these up to units of degrees to match mass_map spacing and make writing the aperture easier
x = x * 0.2/3600
y = y * 0.2/3600

# for now I'll weight everything uniformly
weights = np.ones(len(x))

# Define an NxN grid centered on the cluster
# Spans 1-deg centered at the BCG
# scale so that its in pixel coordinates for computing
N = 51

mid_x = 0
mid_y = 0
x_grid_samples = np.linspace(mid_x-0.5,mid_x+0.5,N)
y_grid_samples = np.linspace(mid_y-0.5,mid_y+0.5,N)
y_grid,x_grid = np.meshgrid(y_grid_samples,x_grid_samples)

In [ ]:
# Using a global R for the first try:

global_R = 0.5 * (np.mean(wl_table['anacal_fpfs_R1']) + np.mean(wl_table['anacal_fpfs_R2']))
print(global_R)

weights = np.ones(len(x))
g1 = wl_table['anacal_fpfs_e1'] / global_R
g2 = wl_table['anacal_fpfs_e2'] / global_R

In [ ]:
plt.hist(wl_table['anacal_fpfs_e1'], bins=100, alpha=0.2, range=[-2, 2], label='e1')
plt.hist(g1, bins=100, alpha=0.2, range=[-2, 2], label='g1 - calibrated')
plt.legend()

In [ ]:
# this takes a minute to run
e_ap,b_ap,v_ap = compute_mass_map(x_grid,y_grid,x,y,g1,g2,weights,schirmer_filter,filter_kwargs={'aperture_size':0.75})

In [ ]:
sky_box = np.array([[x_grid.max(), x_grid.min()],[y_grid.min(), y_grid.max()]])

In [ ]:
test_xs = np.linspace(-5, 5, num=1001)
standard = stats.norm(0,1)
normal_dist = standard.pdf(test_xs)

In [ ]:
plt.hist((b_ap/np.sqrt(v_ap)).flatten(), bins=51, label='B Mode', histtype='step', density=True)
plt.hist((e_ap/np.sqrt(v_ap)).flatten(), bins=51, label='E Mode', histtype='step', density=True)
plt.plot(test_xs, normal_dist, '--', label="Gaussian")
plt.title("FPFS Mass Map Component Distribution")
plt.legend(frameon=False);

In [ ]:
print(np.abs(e_ap/np.sqrt(v_ap)).max(), np.abs(b_ap/np.sqrt(v_ap)).max())

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(10,6))

ax = axs[0]
ax.set_title('E-Mode SN')
MapE = ax.imshow(e_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

ax = axs[1]
ax.set_title('B-Mode SN')
MapB = ax.imshow(b_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

# for xx in axs:
#     xx.plot(ra_bcg, dec_bcg, 'rx')
cbar = fig.colorbar(MapE, ax=axs,fraction=0.025)

fig.suptitle("FPFS Mass Map", y=0.85)
fig.supxlabel("RA", y=0.14)
fig.supylabel("DEC", x=0.05)
#fig.savefig('ACO360_mass_map.png',dpi=480)
# woohoo, we have a signal!

### Weighted R

In [ ]:
def compute_mass_map_anacal(x_grid,y_grid,x,y,e1,e2, R1, R2,
                            weights,q_filter,filter_kwargs={}):
    '''
    This function computes the mass aperture-statistics at each point on a specified grid. Run quality-cuts, NaN filtering, etc. before this step!
    
    Args:
        x: Numpy array; an array of x-coordinates for each object
        y: Numpy array; an array of y-coordinates for each object
        x_grid: Numpy array; an NxM array of x-coordinates to sample the aperture-mass on
        y_grid: Numpy array; an NxM array of y-coordinates to sample the aperture-mass on
        g1; Numpy array; the shear g1 for each object
        g2; Numpy array; the shear g2 for each object
        weights: Numpy array; the weight for each object's shear
        q_filter; function; the filter-function used to compute Map
        kwargs; dict; kwargs passed to w_filter
    
    Returns:
        Map_E: Numpy array; an NxM array containing the E-mode aperture mass evaluated at each grid-point
        Map_B: Numpy array; an NxM array containing the B-mode aperture mass evaluated at each grid-point
        Map_V: Numpy array; an NxM array containing the variance in the aperture mass evaluated at each grid-point

    '''

    y_shape = len(y_grid[:,0])
    x_shape = len(x_grid[0,:])
    
    Map_E = np.zeros((y_shape,x_shape))
    Map_B = np.zeros((y_shape,x_shape))
    Map_V = np.zeros((y_shape,x_shape))
    
    if 'aperture_size' not in filter_kwargs:
        filter_area = np.pi * (8000)**2
    else:
        filter_area = np.pi * filter_kwargs['aperture_size']**2
    
    # an extra catch for an objects assigned NaN g1/g2 just in case
    nan_catch = np.isfinite(e1) & np.isfinite(e2)
    x = x[nan_catch]
    y = y[nan_catch]
    e1 = e1[nan_catch]
    e2 = e2[nan_catch]
    weights = weights[nan_catch]
    
    for i in range(y_shape):
        for j in range(x_shape):
            delta_x = x_grid[j,i] - x
            delta_y = y_grid[j,i] - y
            radius = np.sqrt(delta_x**2 + delta_y**2)# * deg2pix_conv
            theta = np.arctan2(delta_y,delta_x)

            filter_values = q_filter(radius,**filter_kwargs)

            r_weights = (radius < filter_kwargs['aperture_size']).astype(int) # Top-hat
            # r_weights = filter_values # Schrimer weight

            R1s = np.average(R1, weights=r_weights)
            R2s = np.average(R2, weights=r_weights)

            R_val = 0.5*(R1s + R2s)
            # R_val = 0.1

            g1 = e1 / R_val
            g2 = e2 / R_val

            g_T = -g1*np.cos(2*theta) - g2*np.sin(2*theta)
            g_X =  g1*np.sin(2*theta) - g2*np.cos(2*theta)
            g_mag = g1**2 + g2**2

            weight_sum = np.sum(weights)
            
            Map_E[i,j] = np.sum(filter_values*g_T*weights)*filter_area/weight_sum
            Map_B[i,j] = np.sum(filter_values*g_X*weights)*filter_area/weight_sum
            Map_V[i,j] = np.sum( (filter_values**2)*g_mag*(weights**2) )*(filter_area**2)/(2*(weight_sum**2))
    
    return Map_E, Map_B, Map_V

In [ ]:
ra_bcg, dec_bcg = (37.86501659859067, 6.982204815599694)

In [ ]:
# first, setup a coordinates for a flat-sky in x and y
# create a WCS and use the corresponding method to transform sky to px coords

# load wcs from astropy
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel

# also load skycoord for the conversion
from astropy.coordinates import SkyCoord

# build wcs centered on BCG
flat_wcs = WCS(naxis=2)
crval_sky = [ra_bcg*u.deg,dec_bcg*u.deg]
flat_wcs.wcs.crval = [ra_bcg,dec_bcg]
flat_wcs.wcs.crpix = [0,0] # assign 0,0 to the center, shouldn't matter
flat_wcs.wcs.cdelt = [-0.2/3600,0.2/3600] # match angular resolution of LSST, 0.2"
flat_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
flat_wcs.wcs.radesys = 'ICRS'
flat_wcs.wcs.equinox = 2000
flat_wcs.wcs.cd = [[-0.2/3600,0],[0,0.2/3600]]

# compute coords in the flat-sky
# coords = SkyCoord(ra=merged_cat_wl['coord_ra'][source_filt][to_use],dec=merged_cat_wl['coord_dec'][source_filt][to_use],unit='degree')
coords = SkyCoord(ra=wl_table['coord_ra'],dec=wl_table['coord_dec'],unit='degree')
x,y = skycoord_to_pixel(coords,wcs=flat_wcs)

# scale these up to units of degrees to match mass_map spacing and make writing the aperture easier
x = x * 0.2/3600
y = y * 0.2/3600

# for now I'll weight everything uniformly
weights = np.ones(len(x))

# Define an NxN grid centered on the cluster
# Spans 1-deg centered at the BCG
# scale so that its in pixel coordinates for computing
N = 51

mid_x = 0
mid_y = 0
x_grid_samples = np.linspace(mid_x-0.5,mid_x+0.5,N)
y_grid_samples = np.linspace(mid_y-0.5,mid_y+0.5,N)
y_grid,x_grid = np.meshgrid(y_grid_samples,x_grid_samples)

In [ ]:
# # object positions
# # x = merged_cat_wl['coord_ra'][source_filt][to_use]
# # y = merged_cat_wl['coord_dec'][source_filt][to_use]

# x = wl_table['coord_ra']
# y = wl_table['coord_dec']

# # for now I'll weight everything uniformly
# weights = np.ones(len(x))
# # weights = weight

# # Define an NxN grid centered on the cluster
# N = 151

# mid_x = ra_bcg
# mid_y = dec_bcg
# x_grid_samples = np.linspace(mid_x-0.5,mid_x+0.5,N)
# y_grid_samples = np.linspace(mid_y-0.5,mid_y+0.5,N)
# y_grid,x_grid = np.meshgrid(y_grid_samples,x_grid_samples)

In [ ]:
e1 = wl_table['anacal_fpfs_e1']
e2 = wl_table['anacal_fpfs_e2']
R1 = wl_table['anacal_fpfs_R1']
R2 = wl_table['anacal_fpfs_R2']
weights = np.ones(len(wl_table))

In [ ]:
Rmap1 = np.zeros((N, N))
Rmap2 = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        delta_x = x_grid[j,i] - x
        delta_y = y_grid[j,i] - y
        radius = np.sqrt(delta_x**2 + delta_y**2)# * deg2pix_conv
        theta = np.arctan2(delta_y,delta_x)

        filter_values = schirmer_filter(radius, aperture_size=0.4)
        R1s = np.average(R1, weights=filter_values)
        R2s = np.average(R2, weights=filter_values)
        Rmap1[i,j] = 0.5*(R1s + R2s)

        filter_values = (radius < 0.4).astype(int)
        R1s = np.average(R1, weights=filter_values)
        R2s = np.average(R2, weights=filter_values)
        Rmap2[i,j] = 0.5*(R1s + R2s)

In [ ]:
sky_box = np.array([[x_grid.max(), x_grid.min()],[y_grid.min(), y_grid.max()]])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

cmap = cm.coolwarm

ax[0].imshow(Rmap1, vmin=.15, vmax=.35, cmap=cmap, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))
ax[0].set_title("Response using Schirmer Weighting")
# ax[0].plot(wl_table['coord_ra'], wl_table['coord_dec'], 'x', alpha=0.02)

k2 = ax[1].imshow(Rmap2, vmin=.15, vmax=.35, cmap=cmap, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))
ax[1].set_title("Response using Top Hat Weighting")

# for xx in ax:
#     lon = xx.coords[0]
#     lat = xx.coords[1]
    
#     lon.set_major_formatter('d.d')
#     lat.set_major_formatter('d.d')
#     lon.set_axislabel('')
#     lat.set_axislabel('')

fig.supxlabel('RA')
fig.supylabel('DEC', x=0.05)

fig.subplots_adjust(right=0.95)
cbar_ax = fig.add_axes([0.97, 0.12, 0.02, 0.75])
fig.colorbar(k2, cax=cbar_ax)

Weighting is being done very lazily, just change the value of `r_weights` in `compute_mass_map_anacal`

In [ ]:
# this takes a minute to run
e_ap,b_ap,v_ap = compute_mass_map_anacal(x_grid,y_grid,x,y,e1, e2, R1, R2,weights,schirmer_filter,filter_kwargs={'aperture_size':0.6})

In [ ]:
sky_box = np.array([[x_grid.max(), x_grid.min()],[y_grid.min(), y_grid.max()]])

In [ ]:
test_xs = np.linspace(-5, 5, num=1001)
standard = stats.norm(0,1)
normal_dist = standard.pdf(test_xs)

In [ ]:
plt.hist((b_ap/np.sqrt(v_ap)).flatten(), bins=51, label='B Mode', histtype='step', density=True)
plt.hist((e_ap/np.sqrt(v_ap)).flatten(), bins=51, label='E Mode', histtype='step', density=True)
plt.plot(test_xs, normal_dist, '--')
plt.title("Top Hat Weighted")
plt.legend();

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(10,6))

ax = axs[0]
ax.set_title('E-Mode SN')
MapE = ax.imshow(e_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

ax = axs[1]
ax.set_title('B-Mode SN')
MapB = ax.imshow(b_ap/np.sqrt(v_ap),origin='lower',vmax=5,vmin=-2, extent=(sky_box[0,0], sky_box[0,1], sky_box[1,0], sky_box[1,1]))

# for xx in axs:
#     xx.plot(ra_bcg, dec_bcg, 'rx')
cbar = fig.colorbar(MapE, ax=axs,fraction=0.025)

fig.supxlabel("RA", y=0.14)
fig.supylabel("DEC", x=0.05)
fig.suptitle("Top Hat Weighted", y=0.85)
#fig.savefig('ACO360_mass_map.png',dpi=480)

# woohoo, we have a signal!